<a href="https://colab.research.google.com/github/smabb/p/blob/master/Data_loading.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Data Loading, Storage, 

Accessing data is a necessary first step . I’m
going to be focused on data input and output using pandas, though there are numerous
tools in other libraries to help with reading and writing data in various formats.
Input and output typically falls into a few main categories: reading text files and other
more efficient on-disk formats, loading data from databases, and interacting with network
sources like web APIs.

In [0]:
import numpy as np
import pandas as pd
np.random.seed(12345)
import matplotlib.pyplot as plt
plt.rc('figure', figsize=(10, 6))
np.set_printoptions(precision=4, suppress=True)

## Reading and Writing Data in Text Format

Pandas features a number of functions for reading tabular data as a DataFrame
object. Table 6-1 summarizes some of them.

* `read_csv`                 Load delimited data from a file, URL, or file-like object; use comma as default delimiter

* `read_table` Load delimited data from a file, URL, or file-like object; use tab ('\t') as default delimiter

* `read_fwf` Read data in fixed-width column format (i.e., no delimiters)

* `read_clipboard` Version of read_table that reads data from the clipboard; useful for converting tables from web
pages

* `read_excel` Read tabular data from an Excel XLS or XLSX file

* `read_hdf` Read HDF5 files written by pandas

* `read_html` Read all tables found in the given HTML document

* `read_json` Read data from a JSON (JavaScript Object Notation) string 
representation

* `read_msgpack` Read pandas data encoded using the MessagePack binary format

* `read_pickle` Read an arbitrary object stored in Python pickle format
* `read_sas` Read a SAS dataset stored in one of the SAS system’s custom storage formats

* `read_sql` Read the results of a SQL query (using SQLAlchemy) as a pandas DataFrame

* `read_stata` Read a dataset from Stata file format

* `read_feather` Read the Feather binary file format

Handling dates and other custom types can require extra effort. Let’s start with a
small comma-separated (CSV) text file:

In [19]:
%cd examples

/content/examples


Since this is comma-delimited, we can use read_csv to read it into a DataFrame:

In [0]:
df = pd.read_csv('ex1.csv')
df

We could also have used read_table and specified the delimiter:

In [0]:
pd.read_table('ex1.csv', sep=',')

A file will not always have a header row. Consider this file:

In [0]:
!cat ex2.csv

To read this file, you have a couple of options. You can allow pandas to assign default
column names, or you can specify names yourself:

In [0]:
pd.read_csv('ex2.csv', header=None)
pd.read_csv('ex2.csv', names=['a', 'b', 'c', 'd', 'message'])

Suppose you wanted the message column to be the index of the returned DataFrame.
You can either indicate you want the column at index 4 or named 'message' using
the index_col argument:

In [0]:
names = ['a', 'b', 'c', 'd', 'message']
pd.read_csv('ex2.csv', names=names, index_col='message')

In some cases, a table might not have a fixed delimiter, using whitespace or some
other pattern to separate fields. Consider a text file that looks like this:

In [0]:
!cat ex3.txt

While you could do some munging by hand, the fields here are separated by a variable
amount of whitespace. In these cases, you can pass a regular expression as a
delimiter for read_table. This can be expressed by the regular expression \s+, so we
have then:

In [0]:
result = pd.read_table('ex3.txt', sep='\s+')
result

Because there was one fewer column name than the number of data rows,
read_table infers that the first column should be the DataFrame’s index in this special
case.
The parser functions have many additional arguments to help you handle the wide
variety of exception file formats that occur (see a partial listing in Table 6-2). For
example, you can skip the first, third, and fourth rows of a file with skiprows:

In [0]:
!cat ex4.csv
pd.read_csv('ex4.csv', skiprows=[0, 2, 3])

Handling missing values is an important and frequently nuanced part of the file parsing
process. Missing data is usually either not present (empty string) or marked by
some sentinel value. By default, pandas uses a set of commonly occurring sentinels,
such as NA and NULL:

In [0]:
!cat ex5.csv
result = pd.read_csv('ex5.csv')
result
pd.isnull(result)

The na_values option can take either a list or set of strings to consider missing
values:

In [0]:
result = pd.read_csv('ex5.csv', na_values=['NULL'])
result

Different NA sentinels can be specified for each column in a dict:

In [0]:
sentinels = {'message': ['foo', 'NA'], 'something': ['two']}
pd.read_csv('ex5.csv', na_values=sentinels)

## Reading Text Files in Pieces

When processing very large files or figuring out the right set of arguments to correctly
process a large file, you may only want to read in a small piece of a file or iterate
through smaller chunks of the file.
Before we look at a large file, we make the pandas display settings more compact:

In [0]:
pd.options.display.max_rows = 10

In [0]:
result = pd.read_csv('ex6.csv')
result

If you want to only read a small number of rows (avoiding reading the entire file),
specify that with nrows:

In [0]:
pd.read_csv('ex6.csv', nrows=5)

To read a file in pieces, specify a chunksize as a number of rows:

In [0]:
chunker = pd.read_csv('ex6.csv', chunksize=1000)
chunker

The TextParser object returned by read_csv allows you to iterate over the parts of
the file according to the chunksize. For example, we can iterate over ex6.csv, aggregating
the value counts in the 'key' column like so:

In [0]:
chunker = pd.read_csv('ex6.csv', chunksize=1000)

tot = pd.Series([])
for piece in chunker:
    tot = tot.add(piece['key'].value_counts(), fill_value=0)

tot = tot.sort_values(ascending=False)

In [0]:
tot[:10]

## Writing Data to Text Format

Data can also be exported to a delimited format. Let’s consider one of the CSV files
read before:

In [0]:
data = pd.read_csv('ex5.csv')
data

Using DataFrame’s to_csv method, we can write the data out to a comma-separated
file:

In [0]:
data.to_csv('out.csv')
!cat out.csv

Other delimiters can be used, of course (writing to sys.stdout so it prints the text
result to the console):

In [0]:
import sys
data.to_csv(sys.stdout, sep='|')

Missing values appear as empty strings in the output. You might want to denote them
by some other sentinel value:

In [0]:
data.to_csv(sys.stdout, na_rep='NULL')

With no other options specified, both the row and column labels are written. Both of
these can be disabled:

In [0]:
data.to_csv(sys.stdout, index=False, header=False)

You can also write only a subset of the columns, and in an order of your choosing:

In [0]:
data.to_csv(sys.stdout, index=False, columns=['a', 'b', 'c'])

Series also has a to_csv method:

In [0]:
dates = pd.date_range('1/1/2000', periods=7)
ts = pd.Series(np.arange(7), index=dates)
ts.to_csv('tseries.csv')
!cat tseries.csv

## JSON Data

JSON (short for JavaScript Object Notation) has become one of the standard formats
for sending data by HTTP request between web browsers and other applications. It is
a much more free-form data format than a tabular text form like CSV. Here is an
example:

In [0]:
obj = """
{"name": "Wes",
 "places_lived": ["United States", "Spain", "Germany"],
 "pet": null,
 "siblings": [{"name": "Scott", "age": 30, "pets": ["Zeus", "Zuko"]},
              {"name": "Katie", "age": 38,
               "pets": ["Sixes", "Stache", "Cisco"]}]
}
"""

JSON is very nearly valid Python code with the exception of its null value null and
some other nuances (such as disallowing trailing commas at the end of lists). The
basic types are objects (dicts), arrays (lists), strings, numbers, booleans, and nulls. All
of the keys in an object must be strings. There are several Python libraries for reading and writing JSON data. We'll use json here, as it is built into the Python standard
library. To convert a JSON string to Python form, use json.loads:

In [0]:
import json
result = json.loads(obj)
result

json.dumps, on the other hand, converts a Python object back to JSON:

In [0]:
asjson = json.dumps(result)

How you convert a JSON object or list of objects to a DataFrame or some other data
structure for analysis will be up to you. Conveniently, you can pass a list of dicts
(which were previously JSON objects) to the DataFrame constructor and select a subset
of the data fields:

In [0]:
siblings = pd.DataFrame(result['siblings'], columns=['name', 'age'])
siblings

The pandas.read_json can automatically convert JSON datasets in specific arrangements
into a Series or DataFrame. For example:

In [0]:
!cat example.json

The default options for pandas.read_json assume that each object in the JSON array
is a row in the table:

In [0]:
data = pd.read_json('example.json')
data

##  HTML: Web Scraping

Python has many libraries for reading and writing data in the ubiquitous HTML and
XML formats. Examples include lxml, Beautiful Soup, and html5lib. While lxml is
comparatively much faster in general, the other libraries can better handle malformed
HTML or XML files.

Before we scrape a website, we need to take a look at their robots.txt. This file tells us if the website allows scraping or if they do not. To find the robots.txt, type in the base url and add “/robots.txt”. For eg, if we want to crawl shobiddak.com, type in https://www.shobiddak.com/robots.txt at the url box.

If the robots.txt allows full access it says as follows:

User-agent: *

Disallow:

If the robots.txt blocks all access, it contains the following:

User-agent: *

Disallow: /

And if the robots.txt gives partial access, it contains the following, where section stands for the sections that are not to be crawled:

User-agent: *

Disallow: /section/

Import the necessary libraries:
Requests is used in this example to get the html content, BeautifulSoup to parse the html, and pandas to make a dataframe and write to a csv.

In [0]:
import requests 
from bs4 import BeautifulSoup
import pandas

Store the url you want to scrape to a variable. In this example, I go to apartments.com and type in Cincinnati, OH in the search box and press Go as in the screenshot below.

In [0]:
base_url = "https://shobiddak.com/cars?page=1"

Get the html contents from the page. This is done using the requests library

In [0]:
r = requests.get(base_url)
c = r.content

Parse the html. This is done with BeautifulSoup.

In [0]:
soup = BeautifulSoup(c,"html.parser")


 Extract the first and last page numbers. We need to be able to find the first and the last page numbers in order to be able to crawl through all of the pages on the website dynamically. To do this click on the paging section at the end of the page as shown in the screen shot below and right click to inspect.

In [0]:
# To extract the first and last page numbers
#paging = soup.find("div",{"class":"all_contents"}).find("div",{"class":"center-image"}).find("div",{"class":"pagination"}).find_all("a")
paging = soup.find("div",{"class":"all_contents"}).find("div",{"class":"pagination"}).find_all("a")
start_page = paging[1].text
last_page = paging[len(paging)-2].text

Make page links from the page numbers ,crawl through the pages and extract the contents from the corresponding tags. We start a for loop to iterate through each of the pages.

In [0]:
base_url="https://shobiddak.com/cars?page="
for page_number in range(int(start_page),3):
    
    # To form the url based on page numbers
    url = base_url+str(page_number)
    print(url)
    r = requests.get(base_url+str(page_number)+"/")
    c = r.content
    
    #soup = BeautifulSoup(c,"html.parser")

Extract the header class for title and year.Right click on the title and inspect.

In [0]:
# To extract the Title,year,color and price
    placard_header = soup.find_all("div", {"class":"col-xs-4 fixed-box-height guide-card"})
    print(placard_header[0])

Start a for loop to process car by car and extract the values of Car, Color, Year. We extract the individual values by accessing into the inner most tag with the value. 

In [0]:
# To process property by property by looping
    web_content_list=[]
    for item_header in placard_header:
      
      # To store the information to a dictionary
      web_content_dict = {}    
      
      year_color = item_header.find("p",{"class":"second-info"}).find_all("a")
      web_content_dict["Year"] = year_color[0].text
      web_content_dict["Color"] = year_color[1].text      
      web_content_dict["Car"]=item_header.find("p",{"class":"section_title"}).text
      # To store the dictionary to into a list
      web_content_list.append(web_content_dict)

Make a dataframe with the list and write the list to a csv file

In [0]:
df = pandas.DataFrame(web_content_list)
with pd.option_context('display.max_rows', 5, 'display.max_columns', 5,'display.unicode.east_asian_width', True):  # more options can be specified also
    print(df)



# To write the dataframe to a csv file
df.to_csv("Output.csv")

## Interacting with Web APIs

Many websites have public APIs providing data feeds via JSON or some other format.
There are a number of ways to access these APIs from Python; one easy-to-use
method that I recommend is the requests package.
To find the last 30 GitHub issues for pandas on GitHub, we can make a GET HTTP
request using the add-on requests library:

In [0]:
import requests
url = 'https://api.github.com/repos/pandas-dev/pandas/issues'
resp = requests.get(url)
resp

The Response object’s json method will return a dictionary containing JSON parsed
into native Python objects:





In [0]:
data = resp.json()
print(data)

Each element in data is a dictionary containing all of the data found on a GitHub
issue page (except for the comments). We can pass data directly to DataFrame and
extract fields of interest:

In [0]:
issues = pd.DataFrame(data)
issues

## Interacting with Databases

In [0]:
import sqlite3
query = """
CREATE TABLE test
(a VARCHAR(20), b VARCHAR(20),
 c REAL,        d INTEGER
);"""
con = sqlite3.connect('mydata.sqlite')
con.execute(query)
con.commit()

In [0]:
data = [('Atlanta', 'Georgia', 1.25, 6),
        ('Tallahassee', 'Florida', 2.6, 3),
        ('Sacramento', 'California', 1.7, 5)]
stmt = "INSERT INTO test VALUES(?, ?, ?, ?)"
con.executemany(stmt, data)
con.commit()

In [0]:
cursor = con.execute('select * from test')
rows = cursor.fetchall()
rows

In [0]:
cursor.description
pd.DataFrame(rows, columns=[x[0] for x in cursor.description])

In [0]:
import sqlalchemy as sqla
db = sqla.create_engine('sqlite:///mydata.sqlite')
pd.read_sql('select * from test', db)

In [0]:
!rm mydata.sqlite